# Analysis of Artifacts Provided

The artifacts provided are from https://www.malware-traffic-analysis.net Traffic Analysis Exercises with the date of 2020-11-13.  We are using these exercises to provide traininig to analysts on tools and techniques that could be used.  The goal is not to provide a solution for each artifact provided.  Below is a process that can be adapted for each set of artifacts provided.

## Process
1. Download the zip files available for the Exercise
2. Unzip all of the files to the current directory
3. Take precaution this could be real malware that you are analyzing
4. View summaries, notes, text files or pictures showing why you are receiving the artifacts
5. Identify and seperate Internal IP Addresses from External IP Addresses 
6. Identify user accounts, people, machine names related to the Internal IP Addresses
7. Conduct DNS lookups, Virus Total Lookups and whois for External IP Address
8. Hash the files provided and submit the hash to virustotal for additional context (If company policy allows)
9. Evaluate the files with hexdump, strings, debugger or similar tool to identify the functionality of the files
10. Use the pcaps to verify any summary, alerts, context or information found
11. Use Indicators of Compromise confirmed above for additional context
12. Minimize assumptions by reviewing the information, try to only provide facts in your final report

## Global Settings
The below settings apply for all of the code listed below.  This section of the playbook needs to be run first prior to other segments of code.

In [1]:
# Common imports that are necessary
import subprocess
import re
import socket
import struct
import os


# Common variables
currentDir=!pwd
outputDir = currentDir[0] + "/" + "output"
print("Output Directory: " + outputDir)

Output Directory: /home/kali/mta/new/output


## 1. Download the zip files available for Analysis
## 2. Unzip all of the files to the current directory
## 3. Take precaution this could be real malware that you are analyzing

The above steps of the process were conducted manually

## 4. View summaries, notes, text files or pictures showing why you are receiving the artifacts

View the NIDS Alerts file provided

In [2]:
# NIDS Alerts provided
nidsFile = "2020-11-13-traffic-analysis-exercise-alerts.txt"
!cat $nidsFile

## 5. Identify and Separate Internal IP Addresses from External IP Addresses
Evaluate the NIDS alerts

In [3]:
# NIDS Alerts provided
nidsFile = "2020-11-13-traffic-analysis-exercise-alerts.txt"
file = open(nidsFile, 'r')    # open the nidsFile readonly
ipList = []
intIPList = []          # Internal IP List
extIPList = []          # External IP List
for line in file:
        ip = re.findall( r'[0-9]+(?:\.[0-9]+){3}', line )  # Find the ip addresses in the file
        if(ip):                                            # If ip found on the line in the file
            for i in ip:                                   # ip is a list, iterate through the list
                if i not in ipList:                        # If the IP Address is not in the list add it
                    ipList.append(i)
#print("Full List")
#print(ipList)
#print("\n")
print("Internal IP List from the file " + nidsFile)
print("Assumes the internal IP list is 192.168.0.0/16, 10.0.0.0/8 or 172.16.0.0/12")
print("-------------------------------------------------------------------------------------------------")
#print("Converting ip range of 192.168.0.0/16 has a decimal range of 3232235520-3232301055")
#print("10.0.0.0/8 has a decimal range of 167772160-184549375")
#print("172.16.0.0/12 has a decimal range of 2886729728-2887778303")
for ip in ipList:
    packedIP = socket.inet_aton(ip)
    decimalValue = struct.unpack("!L", packedIP)[0]
    if (decimalValue >= 3232235520 and decimalValue <= 3232301055) or (decimalValue >= 167772160 and decimalValue <= 184549375) or (decimalValue >= 2886729728 and decimalValue <= 2887778303):
            print(ip)
    else:
        extIPList.append(ip)
print("\nExternal IP List from the file " + nidsFile)
print("------------------------------------------------------------------------------------------------")
for ip in extIPList:
    print(ip)

Internal IP List from the file 2020-11-13-traffic-analysis-exercise-alerts.txt
Assumes the internal IP list is 192.168.0.0/16, 10.0.0.0/8 or 172.16.0.0/12
-------------------------------------------------------------------------------------------------
192.168.200.8
192.168.200.2

External IP List from the file 2020-11-13-traffic-analysis-exercise-alerts.txt
------------------------------------------------------------------------------------------------
205.185.113.20
143.110.191.95
198.211.99.24
185.141.24.71
198.211.99.168


## 6. Identify user accounts, people, machine names related to the Internal IP Addresses

With the provided resources I am unable to lookup internal IP Addresses, DNS Names, etc.  I would also automate this step if possible.

#### Conducted this step manually
````
192.168.200.2 is the domain controller
192.168.200.8 is the workstation for Craig Alda
- Based on the NIDS alerts from Craig Alda's workstation it conducted zerologon to the domain controller

What is ZeroLogon based on the NIDS? CVE-2020-1472 Allows an unprivileged user to privilege escalate as they authenticate to the domain controller without credentials.

Additional hostnames and information can be pulled from the pcap if necessary...
````

## 7. Conduct DNS lookups, Virus Total Lookups and whois for External IP Address

Using dig, Virustotal and arin to conduct the above information gathering steps

#### Dig

In [4]:
print("Using the above External IP Address List")
print("Conducting dig on the IP Addresses to resolve them to hostnames, domain names, etc.")
for ip in extIPList:
    !dig -x $ip 

Using the above External IP Address List
Conducting dig on the IP Addresses to resolve them to hostnames, domain names, etc.

; <<>> DiG 9.16.2-Debian <<>> -x 205.185.113.20
;; global options: +cmd
;; Got answer:
;; ->>HEADER<<- opcode: QUERY, status: NXDOMAIN, id: 15736
;; flags: qr rd ra; QUERY: 1, ANSWER: 0, AUTHORITY: 0, ADDITIONAL: 1

;; OPT PSEUDOSECTION:
; EDNS: version: 0, flags:; MBZ: 0x0005, udp: 65494
;; QUESTION SECTION:
;20.113.185.205.in-addr.arpa.	IN	PTR

;; Query time: 104 msec
;; SERVER: 192.168.113.2#53(192.168.113.2)
;; WHEN: Mon Dec 14 16:06:56 EST 2020
;; MSG SIZE  rcvd: 56


; <<>> DiG 9.16.2-Debian <<>> -x 143.110.191.95
;; global options: +cmd
;; Got answer:
;; ->>HEADER<<- opcode: QUERY, status: NXDOMAIN, id: 4371
;; flags: qr rd ra; QUERY: 1, ANSWER: 0, AUTHORITY: 0, ADDITIONAL: 1

;; OPT PSEUDOSECTION:
; EDNS: version: 0, flags:; MBZ: 0x0005, udp: 65494
;; QUESTION SECTION:
;95.191.110.143.in-addr.arpa.	IN	PTR

;; Query time: 224 msec
;; SERVER: 192.168.113.2

#### Virustotal
(Whois information can be pulled from Virustotal also)

The below script creates a URL list that can be visited to lookup information about the external IP Addresses

In [5]:
for ip in extIPList:
    print("https://www.virustotal.com/gui/ip-address/" + ip + "/detection\n")

https://www.virustotal.com/gui/ip-address/205.185.113.20/detection

https://www.virustotal.com/gui/ip-address/143.110.191.95/detection

https://www.virustotal.com/gui/ip-address/198.211.99.24/detection

https://www.virustotal.com/gui/ip-address/185.141.24.71/detection

https://www.virustotal.com/gui/ip-address/198.211.99.168/detection



## 8. Hash the files provided and submit the hash to virustotal for additional context (If company policy allows)

Using sha256sum as the program to conduct the hashing of files

In [6]:
listArtifacts = []
for root, subdirectories, files in os.walk(currentDir[0]):
    for file in files:
        if root != currentDir[0] and "ipynb_checkpoints" not in root:
            artifact = os.path.join(root, file)
            if artifact not in listArtifacts:
                listArtifacts.append(artifact)
            checksum = !sha256sum $artifact | awk '{print $1}'
            print("SHA256: " + checksum[0])
            print("URL: https://www.virustotal.com/gui/file/" + checksum[0] + "/detection")
            print("Artifact information using file command: ")
            typeFile = !file $artifact
            print(typeFile[0])
            print("\n\n")
            

SHA256: e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b934ca495991b7852b855
URL: https://www.virustotal.com/gui/file/e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b934ca495991b7852b855/detection
Artifact information using file command: 
/home/kali/mta/new/Windows/System32/Tasks/{5FD47D96-5062-7DE3-08DA-938D00A84B6B}: XML 1.0 document, Little-endian UTF-16 Unicode text, with CRLF line terminators



SHA256: e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b934ca495991b7852b855
URL: https://www.virustotal.com/gui/file/e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b934ca495991b7852b855/detection
Artifact information using file command: 
/home/kali/mta/new/Users/craig.alda/AppData/Roaming/Exijopac/uwsida3/baipuyac.png: PNG image data, 789 x 431, 8-bit/color RGB, non-interlaced



SHA256: e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b934ca495991b7852b855
URL: https://www.virustotal.com/gui/file/e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b934ca495991b7852b855/detection
Artifact information using file command

## 9. Evaluate the files with hexdump, strings, debugger or similar tool to identify the functionality of the files

#### hexdump -C
````
Recommended commands to be executed on the files:
hexdump -C <file>
````

In [7]:
for artifact in listArtifacts:
    print("hexdump -C " + artifact + " | more \n")

hexdump -C /home/kali/mta/new/Windows/System32/Tasks/{5FD47D96-5062-7DE3-08DA-938D00A84B6B} | more 

hexdump -C /home/kali/mta/new/Users/craig.alda/AppData/Roaming/Exijopac/uwsida3/baipuyac.png | more 

hexdump -C /home/kali/mta/new/Users/craig.alda/AppData/Roaming/craig.alda/Maaywuku2.dll | more 

hexdump -C /home/kali/mta/new/Users/craig.alda/AppData/Local/Temp/~2457218.tmp | more 

hexdump -C /home/kali/mta/new/Users/craig.alda/AppData/Local/Temp/sqlite64.dll | more 

hexdump -C /home/kali/mta/new/Users/craig.alda/AppData/Local/Temp/~2559312.dll | more 

hexdump -C /home/kali/mta/new/Users/craig.alda/AppData/Local/Temp/tezehu.exe | more 

hexdump -C /home/kali/mta/new/Users/craig.alda/Documents/CV.xlsb | more 

hexdump -C /home/kali/mta/new/syuHKYt/vFPKnDV/VSMecyU.dll | more 



#### strings
````
strings <file>

Look for functions related to IsDebuggerPresent, CreateProcess, WriteFile, HTTP Functions or TLS functions.  
The above functions indicate malicious characteristics of files.
````

In [8]:
for artifact in listArtifacts:
    print("strings " + artifact + " | more\n")

strings /home/kali/mta/new/Windows/System32/Tasks/{5FD47D96-5062-7DE3-08DA-938D00A84B6B} | more

strings /home/kali/mta/new/Users/craig.alda/AppData/Roaming/Exijopac/uwsida3/baipuyac.png | more

strings /home/kali/mta/new/Users/craig.alda/AppData/Roaming/craig.alda/Maaywuku2.dll | more

strings /home/kali/mta/new/Users/craig.alda/AppData/Local/Temp/~2457218.tmp | more

strings /home/kali/mta/new/Users/craig.alda/AppData/Local/Temp/sqlite64.dll | more

strings /home/kali/mta/new/Users/craig.alda/AppData/Local/Temp/~2559312.dll | more

strings /home/kali/mta/new/Users/craig.alda/AppData/Local/Temp/tezehu.exe | more

strings /home/kali/mta/new/Users/craig.alda/Documents/CV.xlsb | more

strings /home/kali/mta/new/syuHKYt/vFPKnDV/VSMecyU.dll | more



## 10. Use the pcaps to verify any summary, alerts, context or information found

#### Recreate the pcap to only contain IP Addresses from the alarms

#### Evaluates Protocols used by the External IP Addresses
Check for strange protocols and UDP (UDP is not evaluated below outside of DNS)

In [9]:
pcapFile = "2020-11-13-traffic-analysis-exercise.pcap"
print("Protocols used by the External IP Addresses")
print("-------------------------------------------------------------")
print("Count, Source IP, Destination IP, Protocol")
for ip in extIPList:
    !tshark -nr $pcapFile -T fields -e ip.src -e ip.dst -e ip.proto ip.dst=="$ip" | sort | uniq | sort -r | sed 's/6$/TCP/' | sed 's/1$/ICMP/' | sed 's/17$/ICMP/'
print("Completed!")

Protocols used by the External IP Addresses
-------------------------------------------------------------
Count, Source IP, Destination IP, Protocol
192.168.200.8	205.185.113.20	TCP
192.168.200.8	143.110.191.95	TCP
192.168.200.8	198.211.99.24	TCP
192.168.200.8	198.211.99.24	ICMP
192.168.200.8	185.141.24.71	TCP
192.168.200.8	198.211.99.168	TCP
192.168.200.8	198.211.99.168	ICMP
Completed!


#### Evaluating the Destination IP Addresses

In [10]:
print("Destination IP being an External IP Address from the NIDS Log")
print("- Destination Port is less than 1024")
print("- Evaluates TCP ports")
print("-------------------------------------------------------------")
print("Count, Source IP, Destination IP, Destination Port")
for ip in extIPList:
    !tshark -nr $pcapFile -T fields -e ip.src -e ip.dst -e tcp.dstport ip.dst=="$ip" and tcp.dstport le 1024 | sort | uniq -c | sort -rn
print("Completed!")

Destination IP being an External IP Address from the NIDS Log
- Destination Port is less than 1024
- Evaluates TCP ports
-------------------------------------------------------------
Count, Source IP, Destination IP, Destination Port
     67 192.168.200.8	205.185.113.20	80
     60 192.168.200.8	143.110.191.95	443
   3300 192.168.200.8	198.211.99.24	443
      5 192.168.200.8	198.211.99.24	445
    926 192.168.200.8	185.141.24.71	80
      5 192.168.200.8	198.211.99.168	445
Completed!


#### Evaluating the Source IP Address being an External IP of Interest
(Destination port must be less than 1024 to appear)

In [11]:
print("Source IP being an External IP Address from the NIDS Log")
print("- Destination port is less than 1024")
print("- Evaluates TCP ports")
print("-------------------------------------------------------------")
print("Count, Source IP, Destination IP, Destination Port")
for ip in extIPList:
    !tshark -nr $pcapFile -T fields -e ip.src -e ip.dst -e tcp.dstport ip.src=="$ip" and tcp.dstport le 1024 | sort | uniq -c | sort -rn
print("Completed!")

Source IP being an External IP Address from the NIDS Log
- Destination port is less than 1024
- Evaluates TCP ports
-------------------------------------------------------------
Count, Source IP, Destination IP, Destination Port
Completed!


#### DNS Resolution Queries - Narrow Down to Unfamiliar or Suspicious Domains

In [12]:
# Add to the below array the domains that you recognize and KNOW are not involved
commonDNS = [".in-addr.arpa", ".QUIETHUB.NET", ".quiethub.net", "quiethub.net", ".windowsupdate.com", ".msftncsi.com", ".skype.com", ".microsoft.com", "officeapps.live.com", ".bing.com", "login.live.com", ".googleapis.com", ".windows.com", ".oracle.com", ".apple.com", ".msedge.net", ".intel.com", ".digicert.com", ".twitter.com", ".virtualearth.net"]
print("DNS Domain Name Resolutions")
print("-------------------------------------------------------------")
results = !tshark -nr $pcapFile -T fields -e ip.src -e ip.dst -e dns.qry.name dns.qry.name and udp.dstport == 53 | sort | uniq -c | sort -rn
resultsList = []
for line in results:
    match = "no"
    for c in commonDNS:
        if c in line:
            match = "yes"
    if line not in resultsList and match == "no":
        resultsList.append(line)
for line in resultsList:
    print(line)
print("Completed!")

DNS Domain Name Resolutions
-------------------------------------------------------------
    125 192.168.200.8	192.168.200.2	compactmuslimsdeport.pw
      2 192.168.200.8	192.168.200.2	timerdisclaimer.pw
      1 192.168.200.8	192.168.200.2	webintercom76delivery.net
      1 192.168.200.8	192.168.200.2	lezasopedrill.cyou
Completed!


#### Request Method, and URI's accessed by External IP Addresses of Interest from NIDS File

In [13]:
print("Destination IP Address from the NIDS Log with Request method and URI")
print("------------------------------------------------------------------------")
print("Count, SrcIP, DstIP, DstPort, Host, Method, URI")
for ip in extIPList:
    !tshark -nr $pcapFile -T fields -e ip.src -e ip.dst -e tcp.dstport -e http.host -e http.request.method -e http.request.uri ip.dst=="$ip" and http.request.uri | sort | uniq -c | sort -rn
print("Completed!")

Destination IP Address from the NIDS Log with Request method and URI
------------------------------------------------------------------------
Count, SrcIP, DstIP, DstPort, Host, Method, URI
      1 192.168.200.8	205.185.113.20	80	205.185.113.20	GET	/files/3.dll
      1 192.168.200.8	205.185.113.20	80	205.185.113.20	GET	/BVd1qKwd
     77 192.168.200.8	185.141.24.71	80	webintercom76delivery.net	GET	/updates.rss
     14 192.168.200.8	185.141.24.71	80	webintercom76delivery.net	POST	/submit.php?id=123429392
      1 192.168.200.8	185.141.24.71	80	185.141.24.71	GET	/download/winnit.exe
Completed!


#### User Agents used by External IP Addresses of Interest from NIDS File

In [14]:
print("Destination IP Address from the NIDS Log with User Agent")
print("- Look for IP Addresses with more than 1 user-agent")
print("------------------------------------------------------------------------")
print("Count, SrcIP, DstIP, DstPort, Host, Method, URI")
for ip in extIPList:
    !tshark -nr $pcapFile -T fields -e http.host -e http.user_agent ip.dst=="$ip" and http.request.uri | sort | uniq -c | sort -rn
print("Completed!")

Destination IP Address from the NIDS Log with User Agent
- Look for IP Addresses with more than 1 user-agent
------------------------------------------------------------------------
Count, SrcIP, DstIP, DstPort, Host, Method, URI
      2 205.185.113.20	Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 10.0; WOW64; Trident/7.0; .NET4.0C; .NET4.0E; .NET CLR 2.0.50727; .NET CLR 3.0.30729; .NET CLR 3.5.30729)
     91 webintercom76delivery.net	Mozilla/4.0 (compatible; MSIE 8.0; Windows NT 6.1; Trident/4.0)
      1 185.141.24.71	
Completed!


## 11. Use Indicators of Compromise confirmed above for additional context

This step is to be done manually by the analyst.  This also could be automated based on the information above.
````
For example:
Use the uri's detected to find additional IP Addresses that may be involved in the pcap
Use the user-agent strings identified to find additional IP Addresses in the pcap
Use the interesting domain names to find additional activity in the pcap
````

## 12. Minimize assumptions by reviewing the information, try to only provide facts in your final report

Create a report or use this section to automate the creation of a report for passing to others if this appears to actually be an incident.